## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [7]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

In [8]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [10]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [11]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [12]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()

W0811 20:05:49.663001 50244 deprecation_wrapper.py:119] From E:\Users\moon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [13]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [14]:
#training
batch_size = 64
 
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/125
781/781 [==============================] - 529s 678ms/step - loss: 1.8782 - acc: 0.4341 - val_loss: 1.1620 - val_acc: 0.6235
Epoch 2/125
781/781 [==============================] - 525s 672ms/step - loss: 1.3261 - acc: 0.5895 - val_loss: 1.1765 - val_acc: 0.6548
Epoch 3/125
781/781 [==============================] - 525s 673ms/step - loss: 1.1264 - acc: 0.6511 - val_loss: 1.1649 - val_acc: 0.6679
Epoch 4/125
781/781 [==============================] - 524s 671ms/step - loss: 1.0188 - acc: 0.6837 - val_loss: 0.9001 - val_acc: 0.7319
Epoch 5/125
781/781 [==============================] - 524s 671ms/step - loss: 0.9375 - acc: 0.7120 - val_loss: 0.8332 - val_acc: 0.7612
Epoch 6/125
781/781 [==============================] - 524s 671ms/step - loss: 0.8915 - acc: 0.7291 - val_loss: 0.9129 - val_acc: 0.7425
Epoch 7/125
781/781 [==============================] - 524s 671ms/step - loss: 0.8511 - acc: 0.7447 - val_loss: 0.7407 - val_acc: 0.7849
Epoch 8/125
781/781 [====================

781/781 [==============================] - 509s 651ms/step - loss: 0.5875 - acc: 0.8505 - val_loss: 0.6062 - val_acc: 0.8516
Epoch 61/125
781/781 [==============================] - 508s 651ms/step - loss: 0.5861 - acc: 0.8502 - val_loss: 0.5602 - val_acc: 0.8667
Epoch 62/125
781/781 [==============================] - 508s 651ms/step - loss: 0.5816 - acc: 0.8518 - val_loss: 0.6561 - val_acc: 0.8447
Epoch 63/125
781/781 [==============================] - 512s 655ms/step - loss: 0.5871 - acc: 0.8503 - val_loss: 0.5457 - val_acc: 0.8683
Epoch 64/125
781/781 [==============================] - 509s 651ms/step - loss: 0.5813 - acc: 0.8535 - val_loss: 0.6123 - val_acc: 0.8572
Epoch 65/125
781/781 [==============================] - 509s 651ms/step - loss: 0.5850 - acc: 0.8507 - val_loss: 0.6379 - val_acc: 0.8452
Epoch 66/125
781/781 [==============================] - 508s 651ms/step - loss: 0.5802 - acc: 0.8536 - val_loss: 0.6234 - val_acc: 0.8467
Epoch 67/125
781/781 [=========================

781/781 [==============================] - 516s 661ms/step - loss: 0.4032 - acc: 0.8947 - val_loss: 0.4789 - val_acc: 0.8777
Epoch 120/125
781/781 [==============================] - 509s 651ms/step - loss: 0.4047 - acc: 0.8957 - val_loss: 0.4559 - val_acc: 0.8858
Epoch 121/125
781/781 [==============================] - 508s 651ms/step - loss: 0.3988 - acc: 0.8964 - val_loss: 0.4553 - val_acc: 0.8869
Epoch 122/125
781/781 [==============================] - 508s 651ms/step - loss: 0.4012 - acc: 0.8957 - val_loss: 0.4874 - val_acc: 0.8790
Epoch 123/125
781/781 [==============================] - 508s 650ms/step - loss: 0.3979 - acc: 0.8973 - val_loss: 0.4368 - val_acc: 0.8921
Epoch 124/125
781/781 [==============================] - 509s 651ms/step - loss: 0.3972 - acc: 0.8961 - val_loss: 0.4440 - val_acc: 0.8892
Epoch 125/125
781/781 [==============================] - 508s 651ms/step - loss: 0.3919 - acc: 0.8972 - val_loss: 0.4304 - val_acc: 0.8952


In [16]:
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)

10000/10000 [==============================] - 30s 3ms/step


In [18]:
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


Test result: 89.520 loss: 0.430
